In [0]:
df_athletes = spark.read.format("parquet").load("abfss://bronze@olympicsdatalakejess.dfs.core.windows.net/athletes")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
df_athletes.display()

In [0]:
df_athletes = df_athletes.fillna("N/A", subset=["birth_place", "birth_country", "residence_place", "residence_country"])

In [0]:
df_athletes.display()

In [0]:
df_athletes = df_athletes.withColumn("height", col("height").cast(FloatType()))\
    .withColumn("weight", col("weight").cast(FloatType()))


In [0]:
df_athletes.display()


In [0]:
df_athletes = df_athletes.sort(col("height").desc(), col("weight").asc())\
    .filter(col("weight")>0)

df_athletes.display()

In [0]:
df_athletes = df_athletes.withColumn("nationality", regexp_replace(col("nationality"), "United States", "USA"))
df_athletes.display()


In [0]:
df_athletes = df_athletes.withColumnRenamed("code", "athlete_id")
df_athletes.display()

In [0]:
df_athletes = df_athletes.withColumn("occupation", split(col("occupation"), ","))
df_athletes.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df_athletes.withColumn("cum_weight", sum("weight").over(Window.partitionBy("country_code").orderBy(col("weight").desc()).rangeBetween(Window.unboundedPreceding, Window.currentRow)))

In [0]:
df_athletes.columns

In [0]:
df_athletes.display()

In [0]:
df_final = df_athletes.select("athlete_id", "current", "name", "name_short", "name_tv", "gender", "function", "country_code", "country", "country_long", "nationality", "nationality_long", "nationality_code", "height", "weight", "disciplines", "events", "birth_date", "birth_place", "birth_country", "residence_place", "residence_country","lang")

In [0]:
df_final.display()

In [0]:
df_final.write.format("delta").mode("append").option("path", "abfss://silver@olympicsdatalakejess.dfs.core.windows.net/athletes").saveAsTable("olympic_2024.silver.athletes")